In [ ]:
# Check GPU memory, clear and recheck
from numba import cuda
gpu = cuda.select_device(0)
print('Selected GPU is:', gpu)
memory = cuda.current_context().get_memory_info()
print('Memory Status:', memory[0], 'free out of', memory[1], ',', (int(memory[0])/int(memory[1])*100), '% free')

print('Clearing Memory...')
gpu = cuda.select_device(0)
gpu.reset()
memory = cuda.current_context().get_memory_info()
print('Memory Status:', memory[0], 'free out of', memory[1], ',', (int(memory[0])/int(memory[1])*100), '% free')

In [ ]:
# Import Statements
import scipy as sp
import scipy.io as spio
import tensorflow as tf
import matplotlib.pyplot as plt
import time
import math
import numpy as np

from keras import backend as K
from random import randint, uniform, gauss
from math import sqrt
from tqdm import tqdm_notebook, tqdm, tnrange
from pathlib import Path
from pylab import figure, cm
from tensorflow import keras
from matplotlib.colors import LogNorm

# Checking the system is up to date and GPU is enabled
import sys
print(sys.executable)
print(tf.__version__)
K.tensorflow_backend._get_available_gpus()
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())
sess = tf.Session(config=tf.ConfigProto(log_device_placement=True))

In [ ]:
# Load model or create one from scratch
model_path = "BCNN_model.h5"
weights_path = "BCNN_weights.hdf5"
saved_model = False

In [ ]:
# load the data (x is the network input, y is the ground-truth)
n = 1000
x = sp.ndarray((n,32,32,1))
y = sp.ndarray((n,32,32,1))

mat_dict = sp.io.loadmat("./ground_truth.mat")
y[:,:,:,0] = mat_dict["x"]
mat_dict = sp.io.loadmat("./network_input.mat")
x[:,:,:,0]= mat_dict["Y_image"]

# 800 images for training, 100 images for validating, and 100 images for testing
x_train = x[0:800,:,:,:]
y_train = y[0:800,:,:,:]
x_validate = x[800:900,:,:,:]
y_validate = y[800:900,:,:,:]
x_test = x[900:1000,:,:,:]
y_test = y[900:1000,:,:,:]
print('x_train shape is ', x_train.shape)
print('y_train shape is ', y_train.shape)
print('x_validate shape is ', x_validate.shape)
print('y_validate shape is ', y_validate.shape)
print('x_test shape is ', x_test.shape)
print('y_test shape is ', y_test.shape)

In [ ]:
# define the loss functions
import keras_contrib as kc
from keras_contrib.losses import DSSIMObjective
ssObj = DSSIMObjective(k1=0.01, k2=0.03, kernel_size=3, max_value=1.0)
def DSSIM_LOSS(y_true,y_pred):
    mask_true = y_true[:,:,:,0]
    mask_true1 = mask_true[:,:,:,tf.newaxis]
    mask_pred = y_pred[:,:,:,0]
    mask_pred1= mask_pred[:,:,:,tf.newaxis]
    return ssObj(mask_true1,mask_pred1)

def RMSE_LOSS(y_true, y_pred):
    mask_true = y_true[:, :, :, 0]
   #mask_pred = y_pred[:, :, :, 2]
    mask_pred = y_pred[:,:,:,0]
    loss_1 = K.sqrt(K.mean(K.square(mask_true - mask_pred)))
    return loss_1

def mae_loss(y_true, y_pred):
    mask_true = y_true[:, :, :, 0]
    mask_pred = y_pred[:,:,:,0]
    loss_2 = (K.abs(mask_true - mask_pred))
    return loss_2

def bernoulli_loss(y_true, y_pred):
    mask_true = y_true[:,:,:,0]
    mask_pred = y_pred[:,:,:,0]
    loss_3 = K.mean(tf.multiply(mask_true-1,K.log(1-mask_pred + 1e-9)) - tf.multiply(mask_true,K.log(mask_pred + 1e-9)))
    return loss_3

In [ ]:
# construct and compile the network
from keras.models import Model, load_model, Sequential
from keras.layers import Input, Activation
from keras.layers.core import Dropout, Lambda, Dense, Reshape, Permute
from keras.layers.normalization import BatchNormalization
from keras.layers.convolutional import Conv2D, Conv2DTranspose
from keras.layers.pooling import MaxPooling2D
from keras.layers.pooling import AveragePooling2D
from keras.layers.merge import concatenate
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.utils.vis_utils import plot_model
from keras import optimizers
from keras.regularizers import l2

if not saved_model:
    inputs = Input((32,32,1))

    c1 = Conv2D(1, (3, 3), activation='elu', kernel_initializer='he_normal', kernel_regularizer=l2(1e-6), bias_regularizer=l2(1e-6), padding='same') (inputs)
    c1 = Dropout(0.1) (c1)
    #c1 = Dense(16) (c1)
    c1 = Conv2D(16, (3, 3), activation='elu', kernel_initializer='he_normal', kernel_regularizer=l2(1e-6), bias_regularizer=l2(1e-6), padding='same') (c1)
    p1 = MaxPooling2D((2, 2)) (c1)

    c2 = Dropout(0.1) (p1)
    c2 = Conv2D(32, (3, 3), activation='elu', kernel_initializer='he_normal', kernel_regularizer=l2(1e-6), bias_regularizer=l2(1e-6), padding='same') (c2)
    c2 = Dropout(0.1) (c2)
    c2 = Conv2D(32, (3, 3), activation='elu', kernel_initializer='he_normal', kernel_regularizer=l2(1e-6), bias_regularizer=l2(1e-6), padding='same') (c2)
    p2 = MaxPooling2D((2, 2)) (c2)

    c3 = Dropout(0.1) (p2)
    c3 = Conv2D(64, (3, 3), activation='elu', kernel_initializer='he_normal', kernel_regularizer=l2(1e-6), bias_regularizer=l2(1e-6), padding='same') (c3)
    c3 = Dropout(0.1) (c3)
    c3 = Conv2D(64, (3, 3), activation='elu', kernel_initializer='he_normal', kernel_regularizer=l2(1e-6), bias_regularizer=l2(1e-6), padding='same') (c3)
    p3 = MaxPooling2D((2, 2)) (c3)

    c4 = Dropout(0.1) (p3)
    c4 = Conv2D(128, (3, 3), activation='elu', kernel_initializer='he_normal', kernel_regularizer=l2(1e-6), bias_regularizer=l2(1e-6), padding='same') (c4)
    c4 = Dropout(0.1) (c4)
    c4 = Conv2D(128, (3, 3), activation='elu', kernel_initializer='he_normal', kernel_regularizer=l2(1e-6), bias_regularizer=l2(1e-6), padding='same') (c4)
    p4 = MaxPooling2D(pool_size=(2, 2)) (c4)

    c5 = Dropout(0.1) (p4)
    c5 = Conv2D(256, (3, 3), activation='elu', kernel_initializer='he_normal', kernel_regularizer=l2(1e-6), bias_regularizer=l2(1e-6), padding='same') (c5)
    c5 = Dropout(0.1) (c5)
    c5 = Conv2D(256, (3, 3), activation='elu', kernel_initializer='he_normal', kernel_regularizer=l2(1e-6), bias_regularizer=l2(1e-6), padding='same') (c5)

    u6 = Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same') (c5)
    u6 = concatenate([u6, c4])
    c6 = Dropout(0.1) (u6)
    c6 = Conv2D(128, (3, 3), activation='elu', kernel_initializer='he_normal', kernel_regularizer=l2(1e-6), bias_regularizer=l2(1e-6), padding='same') (c6)
    c6 = Dropout(0.1) (c6)
    c6 = Conv2D(128, (3, 3), activation='elu', kernel_initializer='he_normal', kernel_regularizer=l2(1e-6), bias_regularizer=l2(1e-6), padding='same') (c6)

    u7 = Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same') (c6)
    u7 = concatenate([u7, c3])
    c7 = Dropout(0.1) (u7)
    c7 = Conv2D(64, (3, 3), activation='elu', kernel_initializer='he_normal', kernel_regularizer=l2(1e-6), bias_regularizer=l2(1e-6), padding='same') (c7)
    c7 = Dropout(0.1) (c7)
    c7 = Conv2D(64, (3, 3), activation='elu', kernel_initializer='he_normal', kernel_regularizer=l2(1e-6), bias_regularizer=l2(1e-6), padding='same') (c7)

    u8 = Conv2DTranspose(32, (2, 2), strides=(2, 2), padding='same') (c7)
    u8 = concatenate([u8, c2])
    c8 = Dropout(0.1) (u8)
    c8 = Conv2D(32, (3, 3), activation='elu', kernel_initializer='he_normal', kernel_regularizer=l2(1e-6), bias_regularizer=l2(1e-6), padding='same') (c8)
    c8 = Dropout(0.1) (c8)
    c8 = Conv2D(32, (3, 3), activation='elu', kernel_initializer='he_normal', kernel_regularizer=l2(1e-6), bias_regularizer=l2(1e-6), padding='same') (c8)

    u9 = Conv2DTranspose(16, (2, 2), strides=(2, 2), padding='same') (c8)
    u9 = concatenate([u9, c1], axis=3)
    c9 = Dropout(0.1) (u9)
    c9 = Conv2D(16, (3, 3), activation='elu', kernel_initializer='he_normal', kernel_regularizer=l2(1e-6), bias_regularizer=l2(1e-6), padding='same') (c9)
    c9 = Dropout(0.1) (c9)
    c9 = Conv2D(16, (3, 3), activation='elu', kernel_initializer='he_normal', kernel_regularizer=l2(1e-6), bias_regularizer=l2(1e-6), padding='same') (c9)
    
    outputs = Conv2D(1, (3, 3), activation='sigmoid',kernel_initializer='he_normal', kernel_regularizer=l2(1e-6), bias_regularizer=l2(1e-6), padding='same') (c9)


    BCNN_bernoulli_train = Model(inputs=[inputs], outputs=[outputs])

    adm = optimizers.adam(lr=0.0005,decay = (1./0.01-1)/20/500)
    BCNN_bernoulli_train.compile(optimizer=adm, loss=bernoulli_loss)
        
BCNN_bernoulli_train.summary()

In [ ]:
# train the network, and measure the training time
if not saved_model:

    train_start = time.time()
    results = BCNN_bernoulli_train.fit(x_train, y_train, validation_data=(x_validate, y_validate),
                               batch_size=40, epochs=500,verbose=1)  
    train_end = time.time()
    print('Training: time elapsed is', (train_end - train_start))

final_loss = BCNN_bernoulli_train.evaluate(x_test, y_test, batch_size=1)
print('final loss metric is', final_loss)

if not saved_model:
    BCNN_bernoulli_train.save(model_path)
    BCNN_bernoulli_train.save_weights(weights_path)
    print("Saved model:", model_path)

In [ ]:
# Make predictions on testing data, and measure how much time it takes
predict_start = time.time()
prediction = BCNN_bernoulli_train.predict(x_test)
predict_end = time.time()
print('Predicting: time elapsed is', (predict_end - predict_start))
print('Average prediction time was', (predict_end-predict_start)/100*1000, 'milliseconds per image')

In [ ]:
from matplotlib.pyplot import axis as ax
# Visualize a random testing image
test_index = randint(0,100)
print('test index of ',test_index)

inputimage = x_test[test_index,:,:,0] # network input
truth = y_test[test_index,:,:,0] # ground truth
output = prediction[test_index,:,:,0] # network prediction



im1 = plt.matshow(truth, cmap=cm.gray,vmin = 0.0, vmax = 1.0)
cbar1 = plt.colorbar(im1)
plt.gca().xaxis.tick_bottom()
plt.title(r'Ground Truth')

im2 = plt.matshow(output, cmap=cm.gray,vmin = 0.0, vmax = 1.0)
cbar2 = plt.colorbar(im2)
plt.gca().xaxis.tick_bottom()
plt.title(r'Predicted Image')


im3 = plt.matshow(abs(truth-output), cmap=cm.jet,vmin = 0.0, vmax = 1.0)
cbar3 = plt.colorbar(im3)
plt.gca().xaxis.tick_bottom()
plt.title(r'True Absolute Error')

plt.show()

In [ ]:
import scipy.io as sio
Image_BCNN1 = prediction
sio.savemat('Image_BCNN1.mat',{'Image_BCNN1':Image_BCNN1})

In [ ]:
import scipy.io as sio
LOSS_whole = np.asarray(results.history['loss'])
LOSS_val_whole = np.asarray(results.history['val_loss'])
sio.savemat('LOSS_BCNN1.mat',{'LOSS_whole':LOSS_whole})
sio.savemat('LOSS_val_BCNN1.mat',{'LOSS_val_whole':LOSS_val_whole})